# 2) Compute features for random parameters

Using the generated random parameters, we next compute responses and features for 3 different feature sets:

- 'bap': features extracted from somatic recording with 3 different steps + a pulse response measured at the soma plus two locations on the apical dendrite (Backpropagating Action Potential)

- 'soma': features extracted from somatic recording with 3 different steps

- 'extra': features extracted from somatic recording with 3 different steps and from mean extracellular action potential

Computed features are saved in the `config/features/` folder for each parameter set and are ready to be used for optimization.

In [ ]:
%load_ext autoreload
%autoreload

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import numpy as np
import os
import json
from pathlib import Path
from scipy.spatial import distance
import MEAutility as mu
import LFPy
from copy import copy
import time
#import neuroplotlib as npl

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

%matplotlib notebook

In [ ]:
np.random.seed(2308)

In [ ]:
import l5pc_model
import l5pc_evaluator
import l5pc_plot

In [ ]:
save_params = True

## Compute full set of features

In [ ]:
mea_type = 'planar'

In [ ]:
if mea_type == 'linear':
    mea_positions = np.zeros((20, 3))
    mea_positions[:, 2] = 20
    mea_positions[:, 1] = np.linspace(-500, 1000, 20)
    probe = mu.return_mea(info={'pos': list([list(p) for p in mea_positions]), 'center': False, 'plane': 'xy'})
    electrode = LFPy.RecExtElectrode(probe=probe)
elif mea_type == 'planar':
    mea_dim = [50, 20]  # n rows x n cols
    mea_pitch = [35, 35]  # rows and cols pitch

    mea_info = {'dim': mea_dim,
                'electrode_name': 'hd-mea',
                'pitch': mea_pitch,
                'shape': 'square',
                'size': 5,
                'type': 'mea',
                'plane': 'xy'}

    probe = mu.return_mea(info=mea_info)
    # Move the MEA out of the neuron plane (yz)
    probe.move([0, 300, 20])
    # Instantiate LFPy electrode object
    electrode = LFPy.RecExtElectrode(probe=probe)
    
# compute mapping coefficients before


In [ ]:
loaded_params_file = 'config/params/smart_random.csv'
loaded_params = pd.read_csv(loaded_params_file, index_col='index')
loaded_params

In [ ]:
ax = mu.plot_probe(probe)
#npl.plot_neuron(morphology='morphology/C060114A7.asc', ax=ax, plane='xy')

In [ ]:
channels = None

morphology = ephys.morphologies.NrnFileMorphology('morphology/C060114A7.asc', do_replace_axon=True)
param_configs = json.load(open('config/parameters.json'))
parameters = l5pc_model.define_parameters()
mechanisms = l5pc_model.define_mechanisms()

l5pc_cell = ephys.models.LFPyCellModel('l5pc', 
                                       v_init=-65., 
                                       morph=morphology, 
                                       mechs=mechanisms, 
                                       params=parameters)

param_names = [param.name for param in l5pc_cell.params.values() if not param.frozen]      
feature_set = 'all'

print(f'Feature set {feature_set}')
gt_responses = []

if feature_set in ["extra", "all"]:
    fitness_protocols = l5pc_evaluator.define_protocols(electrode) 
else:
    fitness_protocols = l5pc_evaluator.define_protocols() 

if feature_set in ["extra", "all"]:
    sim = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True, electrode=electrode)
else:
    sim = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True)

In [ ]:
responses_1 = {}
params = loaded_params.iloc[0]

In [ ]:
t_start = time.time()
resp_step1 = fitness_protocols['Step1'].run(cell_model=l5pc_cell, param_values=params, sim=sim)
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
resp_step1.keys()

In [ ]:
fig_eap_extra = plt.figure(figsize=(7, 12))
ax_eap_extra = fig_eap_extra.add_subplot(111)
eap = l5pc_evaluator.calculate_eap(responses=resp_step1, protocols=fitness_protocols,
                                   protocol_name='Step1')
eap_norm = eap / np.max(np.abs(eap), 1, keepdims=True)
mu.plot_mea_recording(eap_norm, 
                      probe, colors='C2', ax=ax_eap_extra)#, vscale=2)

ax_eap_extra.set_title("Extra features", fontsize=30)

In [ ]:
probe.positions

In [ ]:
p2v = bpopt.ephys.extra_features_utils.peak_to_valley(eap, 20000)

In [ ]:
neg = bpopt.ephys.extra_features_utils.relative_amplitude(eap)

In [ ]:
pos = bpopt.ephys.extra_features_utils.relative_amplitude(eap, sign='positive')

In [ ]:
ptdiff = bpopt.ephys.extra_features_utils.peak_time_diff(eap, 20000)

In [ ]:
l5pc_plot.plot_feature_map(neg, probe)

In [ ]:
l5pc_plot.plot_feature_map(neg, probe)

In [ ]:
l5pc_plot.plot_feature_map(pos, probe)

In [ ]:
kwargs = {}

In [ ]:
stimulus = fitness_protocols['Step1'].stimuli[0]
kwargs['stim_start'] = stimulus.step_delay
kwargs['stim_end'] = stimulus.step_delay + stimulus.step_duration

In [ ]:
kwargs['recording_names'] = {'': 'Step1.MEA.LFP'}
kwargs['fs'] = 20
kwargs['fcut'] = 1
kwargs['ms_cut'] = [3, 10]
kwargs['upsample'] = 10
kwargs['somatic_recording_name'] = f'Step1.soma.v'
kwargs['channel_locations'] = probe.positions
kwargs['extrafel_feature_name'] = 'neg_peak_diff'

In [ ]:
efeat = bpopt.ephys.efeatures.extraFELFeature(name='ra', exp_mean=0,
                                              exp_std=0, **kwargs)

In [ ]:
ra = efeat.calculate_feature(resp_step1, detect_threshold=0.02)

In [ ]:
l5pc_plot.plot_feature_map(ra, probe, bg='on', norm=True)

In [ ]:
mu.plot_mea_recording(resp_step1['Step1.MEA.LFP']['voltage'], probe)

In [ ]:
plt.figure()
plt.plot(resp_step1['Step1.soma.v']['time'])

In [ ]:
for protocol_name, protocol in fitness_protocols.items():
    print('Running', protocol_name)
    responses_1.update(protocol.run(cell_model=l5pc_cell, param_values=params, sim=sim))

In [ ]:
l5pc_plot.plot_responses(responses_)

In [ ]:
loaded_features = []
loaded_responses = []

for i, (index, params) in enumerate(loaded_params.iterrows()):
    print(f'{i+1} / {len(loaded_params)}, {index}')
    display(params)
    
    feature_folder = f'config/features/{index}'
    _, response, feature_dict = l5pc_evaluator.compute_feature_values(params, l5pc_cell, fitness_protocols, sim, 
                                                                      feature_set=feature_set, probe=probe, 
                                                                      channels=channels,
                                                                      feature_folder=feature_folder,
                                                                      save_to_file=False)
    loaded_features.append(feature_dict)
    loaded_responses.append(response)

In [ ]:
l5pc_plot.plot_multiple_responses(loaded_responses)

## Double check that responses are correct!

In [ ]:
channels = None

morphology = ephys.morphologies.NrnFileMorphology('morphology/C060114A7.asc', do_replace_axon=True)
param_configs = json.load(open('config/parameters.json'))
parameters = l5pc_model.define_parameters()
mechanisms = l5pc_model.define_mechanisms()

l5pc_cell = ephys.models.LFPyCellModel('l5pc', 
                                       v_init=-65., 
                                       morph=morphology, 
                                       mechs=mechanisms, 
                                       params=parameters)

param_names = [param.name for param in l5pc_cell.params.values() if not param.frozen]      
feature_set = 'all'

print(f'Feature set {feature_set}')
gt_responses = []

if feature_set in ["extra", "all"]:
    fitness_protocols = l5pc_evaluator.define_protocols(electrode) 
else:
    fitness_protocols = l5pc_evaluator.define_protocols() 

if feature_set in ["extra", "all"]:
    sim = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True, electrode=electrode)
else:
    sim = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True)

In [ ]:
responses = []

for i, (index, params) in enumerate(random_params.iterrows()):
    print(f'{i+1} / {len(random_params)}, {index}')
    
    feature_folder = f'config/features/{index}'
    _, response, feature_dict = l5pc_evaluator.compute_feature_values(params, l5pc_cell, fitness_protocols, sim, 
                                                                      feature_set=feature_set, probe=probe, 
                                                                      channels=channels,
                                                                      feature_folder=feature_folder,
                                                                      save_to_file=False)
    responses.append(response)

In [ ]:
l5pc_plot.plot_multiple_responses(responses_list=responses)

In [ ]:
l5pc_cell.freeze(params)
l5pc_cell.instantiate(sim=sim)

In [ ]:
for sec in l5pc_cell.LFPyCell.allseclist:
    print(sec, sec.Ra, sec.cm)

In [ ]:
for p in parameters:
    print(p.name, p.value)

### Save random and releas parameters to csv

In [ ]:
if save_params:
    params_folder = Path('config/params')

    if not params_folder.is_dir():
        os.makedirs(params_folder)

    release_params_file = params_folder / 'release.csv'
    random_params_file = params_folder / 'smart_random.csv'

    df_release.to_csv(release_params_file, index_label='index')
    df_random.to_csv(random_params_file, index_label='index')    